# contex-free parsing: pushdown automata

## definition

Context-free grammars (CFGs) and pushdown automata (PDAs) are equivalent in their computational power, meaning that they can recognize and generate the same class of languages: context-free languages.


From CFG to PDA: Given a context-free grammar, we can construct a pushdown automaton that simulates the derivation process of the grammar. 

The PDA uses its **stack** to store and manipulate the non-terminal symbols and productions during the derivation process. 

It guesses the correct production rules non-deterministically to apply and pushes non-terminal symbols onto the stack. 

The PDA then matches the input **non-terminal symbols** with the **terminal symbols** from the production rules and pops them off the stack. 

If input is successfully processed and stack is empty, the PDA accepts the input.

example: a pushdown automaton (PDA) that recognizes the language $L = \{x^n y^n\}$, which consists of strings with n 'x' characters followed by n 'y' characters. 

The PDA uses a stack to keep track of the number of 'x' characters it has encountered and then matches them with the 'y' characters. 

production rules: we could define a CFG for $L = \{x^n y^n\}$

- S → xSy

- S → ε

Here, 'S' is a non-terminal symbol '*'. It represents the start symbol of the grammar and is used to generate strings in the language L by applying the production rules. 

The symbol 'ε' represents an empty string, which indicates that no more production rules need to be applied.

In this grammar, 'x' and 'y' are terminal symbols, while 'S' is a non-terminal symbol.

Here's a step-by-step explanation of the process:

| Step | Action                                      | Stack    | Remaining Input |
|------|---------------------------------------------|----------|----------------|
| 1    | Start with empty stack and input "xxxyyy"  | []       | xxxyyy         |
| 2    | Read 'x', push '*', move to next character | [*]      | xxyyy          |
| 3    | Read 'x', push '*', move to next character | [*, *]   | xyyy           |
| 4    | Read 'x', push '*', move to next character | [*, *, *]| yyy            |
| 5    | Read 'y', pop '*', move to next character  | [*, *]   | yy             |
| 6    | Read 'y', pop '*', move to next character  | [*]      | y              |
| 7    | Read 'y', pop '*', move to end of input    | []       | ""             |


## type

Bottom up: explores options that won’t lead to a full parse 

- shift-reduce (srparser in nltk)

- CKY (Cocke-Kasami-Younger): DP

Top down: explores options that don’t match the full sentence 

- recursive descent (rdparser in nltk)

- Earley parser: DP

Dynamic programming (DP): caches of intermediate results (memoization)

| Method            | Description                                                                                                                                                                                                                                                                           | search method|Pros                                           | Cons                                                                               |
|-------------------|---|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------|-------------------------------------------------------------------------------------|
| Shift-Reduce      | match the RHS of a production until it can build an S. <br>shift operation: push words onto a stack. <br>reduce-n operation: tpop and replace matched words with LHS of production. <br>Stop criteria: when input is processed and S is popped from the stack. | BFS bottom-up| Simple, intuitive                               | left recursion, may generate locally feasible subtrees that are not viable globally   |
| CKY               | chart parser that requires a normalized (binarized) grammar - Chomsky Normal Form (CNF).|DP     bottom-up                                                                       | Efficient due to DP $O(n^3)$| weak equivalence only, syntactic ambiguity |
| Recursive Descent | starts with the start symbol and repeatedly expanding non-terminals. process input left-to-right|DFS   top-down                                                              | Straightforward and easy to implement         | left recursion, may require backtracking, less efficient     |
| Earley Parser     | build a parse forest                                                           | DP top-down|Handles left-recursive and ambiguous grammars | More complicated             |


### Chomsky Normal Form (CNF)

Chomsky Normal Form (CNF) is a specific representation of context-free grammars (CFGs) where all production rules are restricted to one of two forms:

- X → YZ: A non-terminal symbol X produces two non-terminal symbols Y and Z.

- X → w: A non-terminal symbol X produces a single terminal symbol w.

advantage of CNF: simplifies parsing algorithms, e.g. CKY algorithm

All CFG can be converted to CNF, and the resulting CNF grammar generates the same language as the original CFG.

<table>
  <tr>
    <th>Transformation Type</th>
    <th>Description</th>
    <th>Original Rules</th>
    <th>Transformed Rules</th>
  </tr>
  <tr>
    <td>Hybrid Rules</td>
    <td>Split rules with mixed terminal and non-terminal symbols on the RHS</td>
    <td>INF-VP → to VP</td>
    <td>INF-VP → TO VP<br>TO → to</td>
  </tr>
  <tr>
    <td>n-ary Rules</td>
    <td>Transform rules with more than two symbols on the RHS into binary rules</td>
    <td>S → Aux NP VP</td>
    <td>S → R1 VP<br>R1 → Aux NP</td>
  </tr>
  <tr>
    <td>Unary Rules</td>
    <td>Remove rules with a single non-terminal symbol on the RHS</td>
    <td>S → VP<br>VP → Verb<br>VP → Verb NP<br>VP → Verb PP</td>
    <td>S → book<br>S → buy<br>S → R2PP<br>S → VerbPP</td>
  </tr>
  <tr>
    <td>Epsilon Rules</td>
    <td>Remove rules that produce the empty string ε</td>
    <td>S → Verb NP PP &#124; Verb NP<br>NP → book &#124; ε<br>PP → on table &#124; ε<br>Verb → read</td>
    <td>S → Verb NP PP &#124; Verb NP &#124; Verb PP &#124; Verb<br>NP → book<br>PP → on table<br>Verb → read</td>
  </tr>
</table>


## Probabilistic Parsing

### Motivation

a sentence can have multiple possible parses with different probability, need a probabilistic ranking method to choose the most likely parse as final parse for a sentence


### Tasks

- Find most likely parse tree $t$ among all parse trees $T(s)$ that maximizes the probability $p(t)$ given all $n$ production rules

    $$t = \underset{t \in T(s)}{\arg\max}\ p(t) = \underset{t \in T(s)}{\arg\max}\prod_{i=1}^{n} p(\alpha_i \to \beta_i)$$

    possible to do rerank based on specific task (e.g., speech recognition, translation) 
    
    probability of a production rule $(\alpha_i \to \beta_i)$ is estimated using MLE given a training corpus (a set of parsed sentences)

    $$
    p(\beta_i | \alpha_i) = \frac{\text{Count}(\alpha_i \to \beta_i)}{\text{Count}(\alpha_i)}
    $$

    Here, $\text{Count}(\alpha_i \to \beta_i)$ represents the number of times the production rule $(\alpha_i \to \beta_i)$ occurs in the training corpus

    $\text{Count}(\alpha_i)$ represents the total number of times the non-terminal symbol $\alpha_i$ appears on the left-hand side of a production rule in the training corpus.

- compute probability of a sentence $P(s)$:

    If $p(t)$ is the probability of a parse tree $t$, the probability of the sentence $s$ is sum of the probabilities of all possible parse trees $T(s)$ for that sentence:

$$
P(s) = \sum_{t \in T(s)} p(t)
$$

## Lexicalized Parsing

### motivation

Limitations of Probabilistic CFG: Probabilities don't depend on specific words, Cannot disambiguate sentences based on semantics

e.g., the 2 verb phrase below have different parses because different words after 'with'

1. "eat pizza with **pepperoni**": "with pepperoni" describes the pizza

2.  "eat pizza with **fork**": "with fork" describes eat.

<h5>Example Parse tree</h5>
<table>
  <tr>
    <th>Sentence 1: "eat pizza with pepperoni"</th>
    <th>Sentence 2: "eat pizza with a fork"</th>
  </tr>
  <tr>
    <td>
      <pre>
VP (Head: eat)
  V (Head): "eat"
  NP (Head: pizza):
    N (Head): "pizza"
    PP (Head: with) (adjunct):
      P (Head): "with"
      NP (Head: pepperoni):
        N (Head): <span style="color:red;">"pepperoni"</span>
    </td>
    <td>
      <pre>
VP (Head: eat)
  V (Head): "eat"
  NP (Head: pizza):
    N (Head): "pizza"
  PP (Head: with) (adjunct):
    P (Head): "with"
    NP (Head: fork):
      N (Head): <span style="color:red;">"fork"</span>
      </pre>
    </td>
  </tr>
</table>


### limitation

- high Complexity $O(N^5g^3V^3)$

  - $N$ = sentence length
  - $g$ = number of non-terminal symbols
  - $V$ = vocabulary size

  solution: beam search to reduce complexity

- Sparse data problem

  training data is not sufficient to cover all the possible rules and structures that may be encountered during parsing, lead to lower parsing accuracy for previously unseen structures.
  
  e.g., Collins, 40k sentences and 12,409 rules in training, there is still a considerable percentage (15%) of test sentences that contain rules not seen in the training data. 
  
  reason: lexicalized parsing models are more complex, capturing more detailed relationships between words, and thus require more data to accurately learn the underlying structure. 